In [1]:
using StatsBase, Distributions, Plotly, DataFrames, Plots, PyPlot
plotly()
include("episiming.jl");

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9645827951147969916\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4520642831754240650\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9645827951147969916\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /home/rodrigo/.julia/packages/Plots/JKY3H/src/backends.jl:375


Plots.PlotlyBackend()

In [3]:
function powerDecay(a::Number, b::Number, x)
    y = rowWiseNorm(x)
    return 1.0 ./ (1.0 .+ (y ./ a) .^ b)
end

fKernel(x) = powerDecay(1., 1.5, x)

powerDecay (generic function with 1 method)

In [4]:
censoResidencial = [.21, .26, .20, .17, .08, .04, .02, 0.02];

In [5]:
geradorCenso(x) = geraResidencias(x, censoResidencial)
geradorDensidade(k) = (x) -> geraResidencias(x, k)

geradorDensidade (generic function with 1 method)

# Rio de Janeiro

In [ ]:
file = open("landScan.bin", "r")
data = reinterpret(Int64, read(file))
close(file)

dadosRJ = reshape(data, (36, 76));
dadosRJ[dadosRJ .< 0] .= 0;

PyPlot.imshow(log10.(dadosRJ))
PyPlot.colorbar();

In [ ]:
dadosAjustados = floor.(Int, dadosRJ ./ 100);

In [ ]:
@time populacao = distribuiPessoas(1, dadosAjustados, geradorCenso, 100, 700, fKernel);

In [ ]:
# ρForma = 0.2 # shape factor of gamma distribution
# ρEscala = 5 # scale (mean value = scale * shape)
# ρPop = rand(Gamma(ρForma, ρEscala), nPessoas)
αᵣ = 0.8
βᵣ = 0.30
βₛ = 0.10
βᵤ = 0.06
γ = 0.1;

txTransmissaoR = βᵣ ./ [populacao.residencias[i].n ^ αᵣ for i in populacao.pessoas.residencias];
txTransmissaoS = βₛ ./ [populacao.sociais[i].n for i in populacao.pessoas.social];
txTransmissaoG = βᵤ ./ leDistancia(populacao, ones(Bool, populacao.n), ones(Bool, populacao.n), fKernel);

In [ ]:
@time (S, I, R) = evolucaoMista(populacao, Array(1:60), 10, txTransmissaoR, txTransmissaoS, txTransmissaoG, γ, powerDecay);

In [ ]:
Plots.plot((mean(S, dims=1)[1, :]), label = "Suscetíveis")
Plots.plot!((mean(I, dims=1)[1, :]), label = "Infectados")
Plots.plot!((mean(R, dims=1)[1, :]), label = "Recuperados")

In [ ]:
GC.gc()